In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib inline

In [ ]:
import os

In [ ]:
#merge all the log files into one
df = pd.concat( [ pd.read_csv(f, header=None) for f in os.listdir() if f.endswith('csv')])

In [ ]:
df.columns =['throttle','theta','session','image_file']
#df = df[df.session != '05a68148390e41fa89facaff7154103d']

In [ ]:
counts = df.session.value_counts()
counts

In [ ]:
#remove the small sessions as these are normally bad runs
bad_sessions = counts[counts < 90].index

In [ ]:
df = df[~df.session.isin(bad_sessions)]

In [ ]:
df.reset_index(inplace=True)

In [ ]:
image_db = dict()
for f in tqdm_notebook(df.image_file.tolist()):
    image_db[f] = plt.imread(f)

In [ ]:
def fliplr_sequence(seq):
    return np.fliplr( seq.transpose([1,2,3,0])).transpose([3,0,1,2])

In [ ]:
session_idx = df.groupby('session').groups

N= 7
all_ims, all_thetas, all_throttles=[],[],[]
sessions = []

for k in tqdm_notebook(session_idx):
    idx = session_idx[k]
    for j in range(10+N,len(idx)-15):
        #seq = [ images[i] for i in 
        ims = [ df.loc[i].image_file for i in idx[j-N:j] ]
        thetas =  df.loc[idx[j]].theta
        throttles = df.loc[idx[j]].throttle

        #ims = np.array([images[f] for f in ims])


        if len(ims) ==N:
            all_ims.append(ims)
            all_thetas.append(thetas)
            all_throttles.append(throttles)
            sessions.append(k)


In [ ]:
all_seq = pd.DataFrame({'session':k,'theta':all_thetas, 'throttle':all_throttles, 'seq':all_ims})

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_seq, test_seq = train_test_split(all_seq,test_size=.1,random_state=42)

In [ ]:
len(train_seq), len(test_seq)

In [ ]:
def gen_sequence(sequences, batch_size=32, image_db = image_db):
    while True:
        X,Y=[],[]
        for i, row in sequences.sample(batch_size).iterrows():
            seq_ims = row.seq
            theta = row.theta
            ims = np.array([image_db[f] for f in seq_ims])

            if np.random.rand() > .5:
                ims = fliplr_sequence(ims)
                theta =-1*theta

            X.append(ims)
            Y.append(theta)
        yield np.stack(X), np.array(Y)

In [ ]:
X,Y = next(gen_sequence(train_seq, batch_size=1))
fig, ax = plt.subplots(1,N, figsize=(5*N,N))
for k in range(N):
    ax[k].imshow(X[0][k])
print(Y[0])

In [ ]:
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Lambda, BatchNormalization, LSTM, TimeDistributed, GRU, Activation,Dropout
from keras import Model, Sequential
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import multi_gpu_model

In [ ]:
def net():
    inputs = Input(shape=(120,160,3))
    #normalize maybe depends on how the image is loaded
    x = Conv2D(4,(3,3), padding='same', activation=None, kernel_initializer='he_normal')(inputs)
    x= BatchNormalization()(x)
    x = Activation('elu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(8,(3,3), padding='same', activation=None, kernel_initializer='he_normal')(x)
    x= BatchNormalization()(x)
    x = Activation('elu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(16,(3,3), padding='same', activation=None, kernel_initializer='he_normal')(x)
    x= BatchNormalization()(x)
    x = Activation('elu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(32,(3,3), padding='same', activation=None, kernel_initializer='he_normal')(x)
    x= BatchNormalization()(x)
    x = Activation('elu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64,(3,3), padding='same', activation=None, kernel_initializer='he_normal')(x)
    x= BatchNormalization()(x)
    x = Activation('elu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(128,(3,3), padding='same', activation=None, kernel_initializer='he_normal')(x)
    x= BatchNormalization()(x)
    x = Activation('elu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Flatten()(x)
    #x = Dense(64,activation='elu')(x)
    #x = Dense(1,activation='elu')(x)
    
    return Model(inputs=[inputs], outputs=[x])

def lstm_net():
    inputs = Input(shape=(None,256))
    #x = LSTM(128, return_sequences=True, dropout=.1)(inputs)
    x = LSTM(96, return_sequences=False, dropout=.1)(inputs)
    x = Dense(96,activation='elu', kernel_regularizer=l2(1e-4))(x)
    x = Dropout(.2)(x)
    #x = Dense(96,activation='elu')(x)
    #x = Dropout(.1)(x)
    x = Dense(1, activation='tanh')(x)
    return Model(inputs=[inputs], outputs=[x])
    

feature_net = net()
test_net =  lstm_net()


def merged_model():
    inputs = Input(shape=(None,120,160,3))
    x = TimeDistributed(feature_net)(inputs)
    x = test_net(x)
    return Model(inputs=[inputs], outputs=[x])

tnet = multi_gpu_model( merged_model())

opt = Adam(1e-4)
tnet.compile(opt,loss='mean_squared_error')

In [ ]:
tnet.summary()

In [ ]:
tnet.fit_generator(gen_sequence(train_seq,batch_size=64)
                   ,steps_per_epoch=len(train_seq)//64
                   ,epochs=10
                   ,validation_data = gen_sequence(test_seq, batch_size=64)
                   ,validation_steps = len(test_seq)//64)

In [ ]:
feature_net.save('feature_net.h5')
test_net.save('test_lstm.h5')

In [ ]:
X,Y = next(gen_sequence(test_seq, batch_size=1))
fig, ax = plt.subplots(1,N, figsize=(5*N,N))
for k in range(N):
    ax[k].imshow(X[0][k])
print(tnet.predict(X)[0][0], Y[0])